In [1]:
import pickle
X_acl,X_ppg,y,y_participant,X_time = pickle.load(open('../data/tabular_data.p','rb'))

In [2]:
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder
X_ppg = X_ppg[:,np.arange(0,512,2),:]
X_all = np.concatenate([X_ppg,X_acl],axis=-1)

In [3]:
X_all.shape

(43363, 256, 8)

In [5]:
from keras.models import load_model
from sklearn.metrics import accuracy_score
import pandas as pd
filepath = '../model_files/base_cnn_8_seconds_ppg_acl_wesad.hdf5'
index = np.array([0,2,3,4])
model = load_model(filepath)

def get_windows(pred,time,user,n=30,n_per_user = 100):
    df = pd.DataFrame({'time':time,'user':user,'pred':list(pred)})
    X = []
    y = []
    for user in df.user.unique():
        for i in range(n_per_user):
            X.append(np.array([np.array(a) for a in df[df.user==user]['pred'].sample(n,replace=True).values]).reshape(1,n,13))
            y.append(user)
    return np.concatenate(X),np.array(y)

logo = LeaveOneGroupOut()
logo = LeavePGroupsOut(n_groups=1)
for train_index, test_index in logo.split(X_all, y, y_participant.reshape(-1)):
    train_x, test_x_unseen = X_all[train_index], X_all[test_index]
    train_y, test_y_unseen = y[train_index], y[test_index]
    train_participant, test_participant_unseen = y_participant[train_index], y_participant[test_index]
    train_time, test_time_unseen = X_time[train_index], X_time[test_index]
    participant_ids = train_participant.copy()
    train_participant = OneHotEncoder().fit_transform(train_participant.reshape(-1,1)).todense()
    train_x, test_x, train_y, test_y,train_participant, test_participant,train_time,test_time,participant_ids_train, participant_ids_test = train_test_split(train_x,
                                                            train_y,
                                                            train_participant,
                                                            train_time,
                                                            participant_ids,
                                                            test_size = 0.2,
                                                            random_state=42,
                                                            stratify=participant_ids)   
    test_seen_pred = model.predict(test_x[:,:,index])
    test_unseen_pred = model.predict(test_x_unseen[:,:,index])
    print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_seen_pred,axis=1)))
#     X_seen,y_seen = get_windows(test_seen_pred,test_time,participant_ids_test)
#     X_unseen,y_unseen = get_windows(test_unseen_pred,test_time_unseen,test_participant_unseen)
#     y_unseen = np.array([-1]*len(y_unseen))
#     X_total = np.concatenate([X_seen,X_unseen])
#     y_total = list(y_seen)+list(y_unseen)
#     df = pd.DataFrame({'pred':list(X_total),'true':y_total})
    break
    

0.000368052999631947


In [47]:
df['column_mean'] = df['pred'].apply(lambda a:np.mean(a,axis=1))

In [48]:
df['argmax'] = df['column_mean'].apply(lambda a:np.argmax(a))
df['max'] = df['column_mean'].apply(lambda a:np.max(a))

In [49]:
from sklearn.metrics import accuracy_score
# accuracy_score(df['true'],df['argmax'])

0.04466666666666667

In [ ]:
train_x.shape,test_x.shape,val_x.shape,train_participant.shape,test_participant.shape,val_participant.shape, 

In [ ]:
from keras_self_attention import SeqSelfAttention
from keras import backend as K
from keras.layers import Concatenate, Flatten,LeakyReLU, Activation, RepeatVector, Permute, Multiply, Lambda, Dense, merge# Define a regular layer instead of writing a custom layer

In [ ]:
def get_model(input_shape=(256,1),act='relu',loss="categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes,activation='sigmoid',kernel_initializer='normal'))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt,metrics=['acc'])
    return model

In [ ]:
n_classes = len(np.unique(participant_ids_train))
model = get_model(input_shape=(256,3),n_classes=n_classes)
model.summary()

In [ ]:
from keras.models import load_model
filepath = '../model_files/base_cnn_8_seconds_acl_filtered_wesad.hdf5'
# model = load_model(filepath)

In [ ]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_participant,validation_data=(val_x,val_participant), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

In [ ]:
model = load_model(filepath)
test_participant_pred = model.predict(test_x)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered_acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg+filtered_acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg+acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#filtered ppg

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#ppg+acl

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#ppg

In [ ]:
print(confusion_matrix(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
print(accuracy_score(np.argmax(test_participant,axis=1),np.argmax(test_participant_pred,axis=1)))
#acl